<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Post_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [4]:
def extract_files_list(file_list_path):
    files_name = []
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                files_name.append(file_path)
    return files_name

In [5]:
data_files_name = extract_files_list(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")

In [6]:
def make_submission_file_path(file_name):
    return f"{base_path}submissions_{file_name}.json"

In [7]:
import json

def submissions_info(data_files_name):
    submissions_dic = {}
    for file_name in data_files_name:
        file_path = make_submission_file_path(file_name)
        with open(file_path, 'r') as file:
            lines_sub = file.readlines()
        for l in lines_sub:
            obj = json.loads(l)
            if obj['over_18'] == False:
                #obj['media'] == None
                submissions_dic[obj['id']]  = obj
    return submissions_dic

In [8]:
import pandas as pd
import numpy as np
import re
import json

In [9]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations_with_reasoning.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len conversations:", len(conversations))

Len conversations: 32990


In [ ]:
def clean_text(text):
 text = re.sub(r'@[A-Za-z0–9]+', '', text) #Remove @mentions replace with blank
 text = re.sub(r'#', '', text) #Remove the ‘#’ symbol, replace with blank
 text = re.sub(r'RT[\s]+', '', text) #Removing RT, replace with blank
 text = re.sub(r'https?:\/\/\S+', '', text) #Remove the hyperlinks
 text = re.sub(r':', '', text) # Remove :
 return text

In [ ]:
#Next we have to remove emoji & Unicode from the Tweet data.
def remove_emoji(string):
 emoji_pattern = re.compile("["
 u"\U0001F600-\U0001F64F" # emoticons
 u"\U0001F300-\U0001F5FF" # symbols & pictographs
 u"\U0001F680-\U0001F6FF" # transport & map symbols
 u"\U0001F1E0-\U0001F1FF" # flags (iOS)
 u"\U00002500-\U00002BEF" # chinese char
 u"\U00002702-\U000027B0"
 u"\U00002702-\U000027B0"
 u"\U000024C2-\U0001F251"
 u"\U0001f926-\U0001f937"
 u"\U00010000-\U0010ffff"
 u"\u2640-\u2642"
 u"\u2600-\u2B55"
 u"\u200d"
 u"\u23cf"
 u"\u23e9"
 u"\u231a"
 u"\ufe0f" # dingbats
 u"\u3030"
 "]+", flags=re.UNICODE)
 return emoji_pattern.sub(r'', string)

In [ ]:
remove_emoji("hiiii hello (:")

'hiiii hello (:'

In [ ]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS

In [ ]:
def subjectivity(text):
 return TextBlob(text).sentiment.subjectivity
def polarity(text):
 return TextBlob(text).sentiment.polarity

In [ ]:
counter = 0
for key in conversations:
  comments = conversations[key]['comments']
  for comment in comments:
    if comment['reply_to'] is None:
      #it is the original post
      text = remove_emoji(clean_text(comment['body']))
      sub = subjectivity(text)
      pol = polarity(text)
      conversations[key]['subjectivity'] = sub
      conversations[key]['polarity'] =  pol
      break

In [10]:
submissions_info = submissions_info(data_files_name)

In [ ]:
len(conversations)

32990

In [11]:
new_submissions_dict = {}

for key in conversations:
  #print(submissions_info[key]['thumbnail'])
    if not submissions_info[key]['thumbnail'].startswith('https'):
       new_submissions_dict[key] = conversations[key]
# Now, new_submissions_dict contains entries with 'media' as None

In [12]:
len(new_submissions_dict)

18184

In [ ]:
submissions_info['xuk5dd']

{'all_awardings': [],
 'allow_live_comments': False,
 'archived': False,
 'author': 'Neural_Engine_',
 'author_created_utc': 1600795886,
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_7zefqoyh',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_gild': True,
 'category': None,
 'content_categories': None,
 'contest_mode': False,
 'created_utc': 1664805334,
 'discussion_type': None,
 'distinguished': None,
 'domain': 'i.redd.it',
 'edited': False,
 'gilded': 0,
 'gildings': {},
 'hidden': False,
 'hide_score': False,
 'id': 'xuk5dd',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': True,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': False,
 'link_flair

In [13]:
import pickle

with open(f"{base_path}conversations_with_reasoning_media_removed.pkl", 'wb') as file:
    pickle.dump(new_submissions_dict, file)

print("Variables written to the file.")

with open(f"{base_path}conversations_with_reasoning_media_removed.pkl", 'rb') as file:
    loaded_data = pickle.load(file)

print("Len loaded data:", len(loaded_data))


Variables written to the file.
Len loaded data: 18184


In [ ]:
len(new_submissions_dict)

30023

In [ ]:
def extract_files_list(file_list_path):
    files_name = []
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                files_name.append(file_path)
    return files_name

In [ ]:
data_files_name = extract_files_list(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")

In [ ]:
def make_submission_file_path(file_name):
    return f"{base_path}submissions_{file_name}.json"

In [ ]:
subreddits = set()

In [ ]:
def extract_submissions_text_with_filter(data_files_name):
    submissions_dic = {}
    over_18_count = 0
    for file_name in data_files_name:
        file_path = make_submission_file_path(file_name)
        with open(file_path, 'r') as file:
            lines_sub = file.readlines()
        for l in lines_sub:
            obj = json.loads(l)
            if obj['over_18'] == False:
                submissions_dic[obj['id']]  = obj
                subreddits.add(obj['subreddit'])
            else:
                over_18_count += 1
    print("over 18 count ", over_18_count)
    return submissions_dic

In [ ]:
submissions = extract_submissions_text_with_filter(data_files_name)

over 18 count  66876


In [ ]:
#subreddit subscribers number

In [ ]:
c = 0
for s in submissions:
  sub = submissions[s]
  print(sub['author'])
  c+=1

Streaming output truncated to the last 5000 lines.
WanpoBigMara
Pizzalord1
BigLeeWaite
Alternative-Image859
IndependentNet6159
rosetechnology
[deleted]
[deleted]
mahachakravartin
[deleted]
Spacemonster111
bananag2
[deleted]
Rosefier
Vexcenot
Warm_Sundae_6568
[deleted]
PatrickJunk
Fox_1n_The_Box
NewsElfForEnterprise
jenniedajullie
Simsgirl950
matchadd87
weworkatvinted
ST3R10
[deleted]
Snoo-54710
sai_iii
PositiveFail884
kryspypeppers
Complex-Swimmer-9998
AfraidFinance2130
discourtesy
burlap370
Colourblindness
June_Berries
Aware_Veterinarian_3
kelsmichelle
retronoodle
aclu
ZoobBot
bashlad
pestuzikno
Psychological_Item43
SwingBling
[deleted]
virgilrmiller
Baukra
Marglintorianoyat
One_Woodpecker7328
blueiriscat
FlamingoMost
blayzenow
Gyrovague_Greyling
[deleted]
Relevant-Fan144
Thrash2007
ChOoSeUsErNaMe9000
[deleted]
ReallySillyLily36
tabibage
Toe_Itch
T1nyZ
dog_cult_chronicles
[deleted]
[deleted]
NaBUru38
thespuzz
mr2711
IllustriousAd5768
DueWarthog5903
justclau_
MiesM
gwreck5
Robertop213


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to minimize
def objective(x):
    a, b, c, d = x
    return (a + b + c + d - 1)**2  # Constraint: a + b + c + d = 1

# Define the equations as constraints
def constraint1(x):sorted_subreddits = sorted(subreddit_submission_count.items(), key=lambda x: x[1], reverse=True)sorted_subreddits = sorted(subreddit_submission_count.items(), key=lambda x: x[1], reverse=True)

    # Print the top ten subreddits
    print("Top Ten Subreddits:")
    for subreddit, count in sorted_subreddits[:10]:
        print(f"{subreddit}: {count}")

    # Print the top ten subreddits
    print("Top Ten Subreddits:")
    for subreddit, count in sorted_subreddits[:10]:
        print(f"{subreddit}: {count}")
    a, b, c, d = x
    return a*x[0] + b*x[1] + c*x[2] + d*x[3] - (a*x[4] + b*x[5] + c*x[6] + d*x[7])

def constraint2(x):
    a, b, c, d = x
    return a*x[8] + b*x[9] + c*x[10] + d*x[11] - (a*x[12] + b*x[13] + c*x[14] + d*x[15])

# Initial guess for a, b, c, d
x0 = np.array([0.25, 0.25, 0.25, 0.25])

# Define the bounds for a, b, c, d (between 0 and 1)
bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

# Define the constraints
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': constraint1},
        {'type': 'eq', 'fun': constraint2})

# Minimize the objective function with constraints
res = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the result
print(res.x)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to minimize
def objective(x):
    a, b, c, d = x
    return (a + b + c + d - 1)**2  # Constraint: a + b + c + d = 1

# Define the equations as constraints
def constraint1(x):
    a, b, c, d = x
    return a*0.4 + b*0.5 + c*0.1 + d*0.3 - (a*0.5 + b*0.1 + c*0.2 + d*0.7)

def constraint2(x):
    a, b, c, d = x
    return a*0.2 + b*0 + c*0.56 + d*0.9 - (a*0.1 + b*0.8 + c*0 + d*0.4)

# Initial guess for a, b, c, d
x0 = np.array([0.25, 0.25, 0.25, 0.25])

# Define the bounds for a, b, c, d (between 0 and 1)
bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

# Define the constraints
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': constraint1},
        {'type': 'eq', 'fun': constraint2})

# Minimize the objective function with constraints
res = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the result
print(res.x)


[0.24389627 0.32041624 0.23499375 0.20069374]
